In [1]:
from pathlib import Path
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, split, explode
from pyspark.sql.types import StringType, IntegerType, BooleanType, FloatType, TimestampType

DATA_DIR = Path("data/ml-latest-small")

In [2]:
spark = SparkSession.builder\
    .master("local")\
    .appName("Word Count")\
    .getOrCreate()

In [3]:
# TODO: Spark schemas?
def read_df(file: str, types: dict):
    df = spark.read\
        .format("csv")\
        .option("header", "true")\
        .load(str(DATA_DIR / file))
    return set_dtypes(df, types)

def set_dtypes(df, types: dict):
    for column, dtype in types.items():
        dtype = {
            str: StringType(),
            int: IntegerType(),
            float: FloatType(),
            bool: BooleanType(),
            "time": TimestampType() 
        }[dtype]
        df = df.withColumn(column, col(column).cast(dtype))
    return df

df_links = read_df("links.csv", {
    "movieId": int,
    "imdbId": int,
    "tmdbId": int
})
df_movies = read_df("movies.csv", {
    "movieId": int,
    "title": str,
    "genres": str
})
df_ratings = read_df("ratings.csv", {
    "userId": int,
    "movieId": int,
    "rating": float,
    "timestamp": "time"
})
df_tags = read_df("tags.csv", {
    "userId": int,
    "movieId": int,
    "tag": str,
    "timestamp": "time"
})

In [4]:
df_genres = df_movies.select(df_movies.movieId, split(df_movies.genres, "\|").alias("genres"))
df_genres = df_genres.select(df_genres.movieId, explode(df_genres.genres).alias("genre"))

In [5]:
# Search user by id, show the number of movies/genre that he/she has watched
def search_user(user_id: int):
    rated_movies = df_ratings.where(f"userId = {user_id}").select("movieId").distinct()
    tagged_movies = df_tags.where(f"userId = {user_id}").select("movieId").distinct()
    movies = rated_movies.union(tagged_movies).distinct().join(df_movies, on=["movieId"], how="inner")
    movies = movies.select(movies.movieId, explode(split(movies.genres, "\|")).alias("genre"))
    movies = movies.groupBy("genre").count()
    return movies.collect()
search_user(1)

[Row(genre='Crime', count=45),
 Row(genre='Romance', count=26),
 Row(genre='Thriller', count=55),
 Row(genre='Adventure', count=85),
 Row(genre='Drama', count=68),
 Row(genre='War', count=22),
 Row(genre='Fantasy', count=47),
 Row(genre='Mystery', count=18),
 Row(genre='Musical', count=22),
 Row(genre='Animation', count=29),
 Row(genre='Film-Noir', count=1),
 Row(genre='Horror', count=17),
 Row(genre='Western', count=7),
 Row(genre='Comedy', count=83),
 Row(genre='Children', count=42),
 Row(genre='Action', count=90),
 Row(genre='Sci-Fi', count=40)]

In [25]:
# Given a list of users, search all movies watched by each user
def search_movies_by_user(user_ids: [str]):
    rated_movies = df_ratings.filter(df_ratings.userId.isin(user_ids)).select("movieId")
    tagged_movies = df_tags.filter(df_tags.userId.isin(user_ids)).select("movieId")
    movies = rated_movies.union(tagged_movies).join(df_movies, on=["movieId"], how="inner")
    movies = movies.groupBy("title").count()
    return movies.collect()
search_movies_by_user([1, 21])

st Men (1997)', count=1),
 Row(title='Star Wars: Episode VI - Return of the Jedi (1983)', count=2),
 Row(title='Lethal Weapon 4 (1998)', count=1),
 Row(title='Tombstone (1993)', count=1),
 Row(title='Terminator, The (1984)', count=2),
 Row(title='Song of the South (1946)', count=1),
 Row(title='Gladiator (2000)', count=2),
 Row(title='Desperado (1995)', count=1),
 Row(title='JFK (1991)', count=1),
 Row(title='Die Hard: With a Vengeance (1995)', count=1),
 Row(title='Terminal, The (2004)', count=1),
 Row(title='National Treasure (2004)', count=1),
 Row(title='American Pie Presents The Naked Mile (American Pie 5: The Naked Mile) (2006)', count=1),
 Row(title='Horrible Bosses 2 (2014)', count=1),
 Row(title='Wild Things (1998)', count=1),
 Row(title='Fantastic Four (2005)', count=1),
 Row(title='Big Eyes (2014)', count=1),
 Row(title='Teenage Mutant Ninja Turtles III (1993)', count=1),
 Row(title='For Your Eyes Only (1981)', count=1),
 Row(title="Pirates of the Caribbean: At World's End (